In [2]:
import sys
sys.float_info
#Should reveal that floats have 8 bytes

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [3]:
MD = "metaData"
Indep = "trainIndependents"
Dep = "trainDependent"
fileNames = [MD, Indep, Dep]

In [4]:
import math ##For basic mathematical operations
import matplotlib.pyplot as plt
import dask.dataframe as dd  
from IPython import display ## Plot setup for Ipython
from matplotlib import cm ##  Colormap reference
from matplotlib import gridspec ##plot setups
from matplotlib import pyplot as plt ##plot setups
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import collections
import warnings
# warnings.filterwarnings("ignore") - may want to ignore, if know what you are doing, but can be helpful
# warnings.filterwarnings(action = "once") - may want to show only once
# from Queue import priority_queue

In [5]:
#LOAD FRAMES WITH CSV CONTENTS FOR 0: INDEPENDENTS 1: DEPENDENTS
frames = []
for index, fileName in tqdm(enumerate(fileNames)):
    frame = dd.read_csv("./TestData/" + fileName + ".csv", sep=",", sample = 2000000)
    if(fileName != 'metaData'):
        frames.append(frame)

3it [02:25, 48.63s/it]


In [6]:
#HANDLES NAs ! 
#about 8 seconds
for i in tqdm(range(len(frames))):
    frames[i] = frames[i].fillna(0)

100%|██████████| 2/2 [00:02<00:00,  1.38s/it]


In [7]:
X = frames[0].iloc[:, 1:].to_dask_array(lengths=True) 
y = frames[1].iloc[:, 1:].to_dask_array(lengths=True)

X = np.array(X)
y = np.array(y)

#data split, difference between row indices yields the number of TIMES included in the dataset
xtrain = X[14:16, :]
ytrain = y[14:94, :]

xtest = X[94:114, :]
ytest = y[94:114, :]

[[-0.00289249 -0.00289249 -0.00289249 ...  0.          0.
   0.        ]
 [ 0.00257159 -0.00190301 -0.00190301 ...  0.          0.
   0.        ]]


In [ ]:
NUM_INDEPENDENT = 20
NUM_SYMBOLS = 1154
#how many times "20070103_13500" to include in building the symbol training sets, must be precise to avoid training on extra zeros
NUM_TIMES = 2

symbol_training_sets = np.zeros(shape=(NUM_SYMBOLS, NUM_TIMES), dtype = tuple)

for i in range(len(xtrain)):
    for j in range(NUM_SYMBOLS):
        independents = []
        #print(independents)
        for k in range(NUM_INDEPENDENT):
            independents.append(xtrain[i][j*NUM_INDEPENDENT + k])     
        symbol_training_sets[j,i] =(independents, ytrain[i][j])

symbol = 0
#prints a training set for the given symbol
print(symbol_training_sets[symbol])



2


In [11]:
for i in range(len(xtrain[0])):
    print(xtrain[0][i])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices